In [1]:
import pandas as pd, numpy as np

In [2]:
# !pip install -q "transformers==4.5.1"
# !pip install -q yattag
# !pip install -q sentencepiece 
# !pip install -q datasets
# !pip install jsonlines

In [4]:
import numpy as np
import torch
from IPython.core.display import display, HTML
from transformers import BertTokenizer

from lrp_bert_modules import LRPBertForSequenceClassification
from heatmap import html_heatmap

In [5]:
model = "mbert"

In [7]:
from transformers import BertTokenizerFast
import os

from transformers import (
    # AutoTokenizer, 
    AutoModel,
    AutoModelForSequenceClassification, 
    AutoConfig,
    CamembertTokenizer,
    BertTokenizer,
    BertTokenizerFast,
    BertConfig,
    XLMRobertaTokenizer,
    XLMRobertaTokenizerFast,
    XLMRobertaConfig,
)

config = BertConfig.from_pretrained('bert-base-multilingual-cased')
config.num_labels = 4

tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')


In [ ]:
print("Loading model...")


state_dict_path = os.path.join("../../results", 'mbert.pt')
state_dict = torch.load(state_dict_path)

# pooler layer?? https://github.com/google-research/bert/issues/1102
model = LRPBertForSequenceClassification(config)
model.load_state_dict(state_dict, strict=False)
model.eval()
print("Done.")

In [8]:
from datasets import load_dataset, list_metrics, load_dataset, Dataset
DATASET_METADATA = {
    'wisesight_sentiment': {
        'huggingface_dataset_name': 'wisesight_sentiment',
        # 'task': Task.MULTICLASS_CLS,
        'text_input_col_name': 'texts',
        'label_col_name': 'category',
        'num_labels': 4,
        'split_names': ['train', 'validation', 'test']
    }
}
dataset = load_dataset(DATASET_METADATA["wisesight_sentiment"]["huggingface_dataset_name"])

Dataset wisesight_sentiment downloaded and prepared to C:\Users\Tk\.cache\huggingface\datasets\wisesight_sentiment\wisesight_sentiment\1.0.0\caa11b92fd29eb91216ff65874c33dffe7d18fff564b073bbf60f2c92693c419. Subsequent calls will reuse this data.


In [ ]:
from scipy.special import softmax
    
def calLRP(model, tokenizer, s):
    inputs = tokenizer(s, return_tensors="pt")
    model.attr()
    output = model(**inputs)

    o = {}
    o["text"] = s

    input_ids = inputs.input_ids.detach().cpu().numpy()[0]

    rel_y = np.zeros(output.shape)
    rel_y[:, 1] = output[:, 1]
    rel_word, rel_pos, rel_type, rel_embed = model.attr_backward(rel_y, eps=.1)
    # print(rel_word, rel_word.shape)
    # rel_word = np.sum(rel_word[0, 1:-1], -1)
    # rel_pos = np.sum(rel_pos[0, 1:-1], -1)
    # rel_type = np.sum(rel_type[0, 1:-1], -1)
    # rel_embed = np.sum(rel_embed[0, 1:-1], -1)

    o["tokens"] = tokenizer.convert_ids_to_tokens(input_ids)
    o["output"] = output.tolist()
    o["lrp_word"] = rel_word.tolist()
    o["lrp_pos"] = rel_pos.tolist()
    o["lrp_type"] = rel_type.tolist()
    o["lrp_embed"] = rel_embed.tolist()

    return o

# test_example = "ดูซิแมวตัวนั้นน่ารักมากกกกกกก"
# model.eval()
# o = calLRP(model, tokenizer, test_example)
# output = o["output"]
# print("Attr Forward Pass Output:")
# print(output)

from tqdm.notebook import tqdm
import jsonlines
import json
model.cuda()
model.eval()
raw_data = []

for i, t in tqdm(enumerate(dataset["validation"])):
    o = calLRP(model, tokenizer, t["texts"])
    raw_data.append(o)

    if i%10==0:

    with jsonlines.open(f"../../lrp-mbert-validation{i}.jsonl", 'w') as writer:
        writer.write_all(raw_data)
    break